In [132]:
spacex_df3 = spacex_df[(spacex_df['Launch Site'] == 'CCAFS LC-40')]
df3 = spacex_df3[spacex_df3['Payload Mass (kg)']  <= 1000]
df4 = df3[df3['Payload Mass (kg)']  >= 10]
fig2 = px.scatter(data_frame=df4,x='Payload Mass (kg)',y='class',color='Booster Version Category')
# fig = px.scatter(data_frame=spacex_df3,x='Payload Mass (kg)',y='class',color='Booster Version Category')
fig2.show()
# spacex_df

In [136]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(
                                    options=[{'label': 'All Sites', 'value': 'ALL'},
                                             {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                             {'label': 'KSC LC-39A','value':'KSC LC-39A'},
                                             {'label': 'VAFB SLC-4E','value':'VAFB SLC-4E'},
                                             {'label': 'CCAFS SLC-40','value':'CCAFS SLC-40'}
                                            ],
                                    value='ALL',
                                    placeholder="ehter your site here",
                                    searchable=True,
                                    id = 'site-dropdown'
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                    min=0, max=10000, step=1000,
                                    marks={0: '0',
                                           2500:'2500',
                                           5000:'5000',
                                           7500:'7500',
                                           10000: '10000'},
                                    value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback([Output(component_id='success-pie-chart', component_property='figure'),
               Output(component_id='success-payload-scatter-chart', component_property='figure')],
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id="payload-slider", component_property="value")])
def get_pie_chart(*entered_site):
#     print(entered_site)
    filtered_df = spacex_df
    if entered_site[0] == 'ALL':
        df3 = spacex_df[spacex_df['Payload Mass (kg)']  <= entered_site[1][1]]
        df4 = df3[df3['Payload Mass (kg)']  >= entered_site[1][0]]
        fig2 = px.scatter(data_frame=df4,x='Payload Mass (kg)',y='class',color='Booster Version Category')
        spacex_df2 = spacex_df[spacex_df['class'] == 1]
        val = spacex_df2['Launch Site'].value_counts()
        fig = px.pie(val, values=dict(val).values(), 
        names=dict(val).keys(), 
        title='All sites')
        return [fig,fig2]
    else:
        spacex_df3 = spacex_df[(spacex_df['Launch Site'] == entered_site[0])]
        df3 = spacex_df3[spacex_df3['Payload Mass (kg)']  <= entered_site[1][1]]
        df4 = df3[df3['Payload Mass (kg)']  >= entered_site[1][0]]
        fig2 = px.scatter(data_frame=df4,x='Payload Mass (kg)',y='class',color='Booster Version Category')
        spacex_df2 = spacex_df[spacex_df['Launch Site'] == entered_site[0]]
        val = spacex_df2['class'].value_counts()
        fig = px.pie(val, values=dict(val).values(), 
                names=[0,1], 
                title=entered_site[0])
        return [fig,fig2]
      
# TASK 4
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output


# Run the app
if __name__ == '__main__':
    app.run_server()